# <u>What is the flow?</u>

The user asks or says something.
* Non-agentic: Encode the user input, retrieve the top results from the vector DB, pass it into the LLM, and then the LLM responds with the given context. For even more control, I can prompt the user with dropdown menus for metadata, although at this point, it's really limiting the deep learning application.
* Agentic: The LLM is an agent. It will figure out what it should do - it can respond immediately or it will take the user input and format it into something else. For example, the LLM retrieves the important information from the user input - genre, ratings threshold, author, etc. Then, the LLM creates a new text from the important information, encodes the new text, and queries the vector DB, along with metadata filters.
* Semi-agentic: I prompt the LLM with something like "You are a book recommender. If the user requests something specific, extract the key information from the user request in this format: {"genre": some_genre, "author": some_author, etc.}. If the user request does not contain a field, like "author", do not include it in the dictionary." The LLM pulls the key information, and then I encode the user input and query the database with the metadata. So the LLM is not an agent, but it helps with parsing the key information from the user input.

How do I include weighting by number of ratings and rating?

Let's start from low complexity and build our way up.

On the machine learning side:
* 1: Embed user query, retrieve, feed to chatbot. Flask/Fast and Streamlit locally hosted. Where to store conversation history? I don't need backend with Streamlit?
* 2: Embed user query, retrieve, feed to chatbot. Containerize and deploy using AWS Lambda and AWS App Runner.
* 3: Use LLM to extract information from user query for metadata filtering via prompting. Reformat user query, embed, retrieve, feed to chatbot. If user query is not a true query, do not retrieve. Create tools for agentic behavior. Like a function that returns available genres.
* 4: Fine-tune smaller model to extract information from user query. Use LLM to provide training samples.

On the deployment side:

1. Use Streamlit to locally host and to deploy to cloud. If you're only using OpenAI's API for retrieval and generation, you don't need more than this.
2. Add Flask or fastAPI for the backend and integrate with Streamlit.
3. Containerize your code, which includes Flask or fastAPI, the retrieval code, the chatbot code, and deploy to EC2 instance.
4. Deploy code using 
5. Containerize the retrieval agent and the chatbot and deploy to EC2 instance. 
6. Use Flask and fastAPI for the backend

Book recommendation system using RAG and OpenAI embedding model and chat completion. Flow: User query -> (optional) intent classifier -> (optional) extract information from user query -> embed user query -> retrieve relevant books from vector DB -> pass user query and book information to chatbot -> chatbot responds and updates conversation history.

Implementation plan, from low to high complexity:

1. Directly embed user query and retrieve (no extraction of key information; embed the user query as-is). Conversation is stored in `st.session_state` (only persists for the duration of the user's session). Use Streamlit to deploy (both locally and to Streamlit Cloud).
    * Deployed locally; Streamlit Cloud failed and now it won't even let me try to create the app
2. Before retrieval, use LLM with prompting to extract key information from the user query and reformat user query before embedding. Add metadata filtering to retrieval.
3. Add fastAPI for the backend.
4. Containerize the code (fastAPI, Streamlit, OpenAI API calls) and deploy to EC2 instance.
5. Containerize the backend and deploy using AWS Lambda. Deploy Streamlit using Streamlit Cloud, calling AWS Lambda functions via API Gateway.
6. Add intent classifier as the first step to classify user query into relevant and non-relevant. Relevant queries are requests for book recommendations; all others are non-relevant. Use LLM to generate synthetic data, then fine-tune the encoder model (like BERT) for intent classification, using MLFlow and AWS SageMaker. For non-relevant queries, skip directly to chatbot (no retrieval).
    * Completely new, unrelated query (retrieval)
    * Follow-up query to the previous query (no retrieval)
    * New, related query, e.g. asking for similar books to the previous results (retrieval)
    * Non-relevant query (no retrieval)
7. Add NER classifier to extract key information from the user query and reformat user query before embedding (this replaces the LLM with prompting from step 2). Use LLM to generate synthetic data, then fine-tune encoder model for NER classification, using MLFlow and AWS SageMaker.
8. Note that loading these models for each Lambda will increase latency.
9. For long sessions, reduce conversation history length via summarization or some other method.
10. To maintain conversation history across sessions, store conversation history to AWS DynamoDB.
11. Query expansion?
12. Re-ranking?
13. Add LangChain at some point
14. CI/CD with GitHub Actions. Unittest and pytest libraries. Streamlit recommended using pytest to test changes.
15. Use pip instead of conda
15. Kubernetes?
* Stress test concurrent users for backend
* Profile app speed with and without asynchronous programming
16. Stretch goal - customize Streamlit frontend with HTML/CSS

# <u>Waqas recommendations</u>
* Backend Enhancements: Use AWS Lambda for backend implementation to gain experience with serverless architectures. I got Lambda working (synchronous invocation through API Gateway).
    * DynamoDB to store user chat history
* Can I add Nginx to my stack (server-based architecture)? https://github.com/luntaixiax/wcd-web-nginx
* NLP Training and Deployment: Utilize SageMaker for training NLP systems, such as classification or Named Entity Recognition (NER), and also for deploying these models.
* Open-Source Models: Experiment with open-source models like Hugging Face's 1B, 3B, or 7B parameters for text generation tasks.
* Embedding Models: Leverage any open-source embedding models to enrich recommendations or search functionalities.
* Advanced Recommendation Systems: Instead of simple content-based recommendations, incorporate algorithms like collaborative filtering or hybrid approaches to make the system more sophisticated.
    * Keyword search
    * Graph-based
* Search Functionality: Implement a fallback mechanism to search the internet if the requested information is not available in your data.
* Experiment Tracking: Use MLFlow to log and track your experiments for better reproducibility and monitoring.
* CI/CD Pipelines: As you already plan to implement CI/CD pipelines, ensure it is robust and scalable.

Additional Features to Explore
* Computer Vision: Allow users to upload book cover images, and build a system to recommend similar books based on the cover.
* Speech Capabilities: Integrate speech-to-text and text-to-speech functionalities so users can interact with your system through voice commands, and the system can respond audibly.

# <u>Next steps</u>
* Streamlit Cloud deployment fails. Packages install but then it complains about ModuleNotFoundError. Go through the logs and see if you can figure it out.
* <s>Successfully deployed containerized Streamlit app to EC2; accessed via web browser</s>
* <s>Create FastAPI backend for your app</s>
    * With FastAPI, profile your code to determine performance (speed)
    * <s>`lifespan` with single instance of `BookRetriever` and `BookAssistant` and `Depends` to inject</s>
    * Websockets
    * <s>`async`</s>
* <s>Create Streamlit frontend to pair with FastAPI app</s>
* <s>Containerize frontend and backend using Docker Compose. Use pip for package management. Deploy to EC2.</s>
* Unittest/pytest + GitHub Actions
* metadata filtering
* Can I use regex with metadata filtering?
* Should my backend return a status code of 200 as well?

# <u>Questions</u>
* HEALTHCHECK - even though my container is running fine, status is unhealthy
* Push multiple images to one ECR repository?
* Should I create a BookAssistant for each user? Or would it be sufficient to create one BookAssistant with async methods? Calling OpenAI and Qdrant APIs asynchronously? Websockets?

```python
backend/
├── __init__.py
├── dependencies.py     # Define dependency functions for BookAssistant, BookRetriever
├── main.py             # Instantiate FastAPI with lifespan function. Lifespan function will initialize BookAssistant, BookRetriever.
├── models
│   ├── __init__.py
│   ├── assistant.py    # Define BookAssistant class. For methods that call OpenAI API, use async/await. One instance will be shared across all users; simply update messages_list per user. Store convo histories in dict.
│   └── retriever.py    # Define BookRetriever class. For methods that call Qdrant/OpenAI APIs, use async/await. One instance will be shared across all users.
├── routers
│   ├── __init__.py
│   ├── chat.py         # One path operation function for both retrieval and chatting. Inject BookAssistant and BookRetriever objects and use Websocket for client-server communication.
│   └── retrieve.py     # Deprecated
└── services            # Don't need this
    ├── __init__.py
    └── vectordb.py
```

```graphql
project/
│
├── backend/
│   ├── __init__.py
│   ├── main.py          # Entry point for FastAPI application
│   ├── models/          # Chatbot and data-related classes
│   │   ├── __init__.py
│   │   ├── chatbot.py   # Chatbot class definition
│   │   └── embedding.py # Query embedding utilities
│   ├── services/        # Logic to interact with the vector DB and OpenAI API
│   │   ├── __init__.py
│   │   ├── vector_db.py # Interfacing with vector DB
│   │   ├── openai_api.py # Interfacing with OpenAI APIs
│   └── routers/         # API routes
│       ├── __init__.py
│       ├── retrieve.py  # Retrieval logic
│       └── chat.py      # WebSocket logic
│
├── frontend/            # Optional: Streamlit or React/JS frontend
│   ├── app.py           # Streamlit application
│
├── tests/               # Unit and integration tests
│   ├── test_retrieve.py
│   ├── test_chat.py
│   └── ...
└── requirements.txt     # Python dependencies
```
